In [126]:
import pandas as pd
import numpy as np

dataset_dir = "../datasets/ml-25m"

movies_df = pd.read_csv(dataset_dir + "/movies.csv")
movies_df = movies_df.set_index("movieId")
ratings_df = pd.read_csv(dataset_dir + "/ratings.csv")
tags_df = pd.read_csv(dataset_dir + "/tags.csv")

print('Setup complete!')

Setup complete!


In [127]:
genres_list = []
for index, row in movies_df.iterrows():
    try:
        genres = row['genres'].split("|")
        genres_list.extend(genres)
    except:
        genres_list.append(row["genres"])

genres_list = list(set(genres_list))

In [128]:
genres_list.remove("IMAX")
repl_idx = genres_list.remove("(no genres listed)")
for genre in genres_list: 
    print(genre)
print(len(genres_list))

Musical
Comedy
War
Mystery
Romance
Children
Sci-Fi
Adventure
Crime
Western
Animation
Horror
Fantasy
Action
Documentary
Thriller
Film-Noir
Drama
18


In [129]:
movies_df1 = movies_df.copy()
for genre in genres_list:
    movies_df1[genre] = movies_df1["genres"].map(lambda o: 1 if genre in str(o) else 0)
movies_df1

,title,genres,Musical,Comedy,War,Mystery,Romance,Children,Sci-Fi,Adventure,Crime,Western,Animation,Horror,Fantasy,Action,Documentary,Thriller,Film-Noir,Drama
movieId,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,1,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0
2,Jumanji (1995),Adventure|Children|Fantasy,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0
3,Grumpier Old Men (1995),Comedy|Romance,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
5,Father of the Bride Part II (1995),Comedy,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209157,We (2018),Drama,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
209159,Window of the Soul (2001),Documentary,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
209163,Bad Poems (2018),Comedy|Drama,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [130]:
movies_df

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
...,...,...
209157,We (2018),Drama
209159,Window of the Soul (2001),Documentary
209163,Bad Poems (2018),Comedy|Drama


In [131]:
tags_df["tag"] = tags_df["tag"].map(lambda t: str(t).lower()) #all tags to small letters
tag_counts = tags_df["tag"].value_counts().rename_axis('tag').reset_index(name='counts')
tag_counts = tag_counts.groupby("tag").counts.agg([sum]) #merge duplicates
tag_counts

,sum
tag,
alexander skarsgård,1
ballet school,1
breakup,1
difficult to find it,1
filmes antigos,2
...,...
淘金记 莫声版,1
独闯龙潭,1
臥底,1


In [132]:
tag_counts = tag_counts[tag_counts["sum"] >= 20]
tag_counts = tag_counts.sort_values(["sum","tag"], ascending=[False,True]).reset_index()
tag_counts

,tag,sum
0,sci-fi,8795
1,atmospheric,7053
2,action,6783
3,comedy,6366
4,surreal,5584
...,...,...
5773,william holden,20
5774,women's lib,20
5775,worst movie ever,20
5776,writers at work,20


In [133]:
init_tag = np.zeros(len(movies_df))
movies_df2 = movies_df.copy()

def add_tags_as_features(tag):
    relevant_tags.add(tag)
    movies_df2[tag] = init_tag
    
relevant_tags = set()
tag_counts["tag"].apply(add_tags_as_features)
    

0       None
1       None
2       None
3       None
4       None
        ... 
5773    None
5774    None
5775    None
5776    None
5777    None
Name: tag, Length: 5778, dtype: object

In [134]:
movies_df2 = movies_df2.drop(["genres", "title"], axis=1)
movies_df2

,sci-fi,atmospheric,action,comedy,surreal,funny,based on a book,twist ending,visually appealing,romance,...,walked out/didn't finish,warlock,watch-again,well-written,william friedkin,william holden,women's lib,worst movie ever,writers at work,yahoo top pick
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
209159,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
209163,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [135]:
#around 10 minutes

def add_tags_to_movies(row):
    tag = row["tag"]
    if tag not in relevant_tags:
        return
    movieId = row["movieId"]
    movies_df2.loc[movieId, tag] += 1
    
tags_df.apply(add_tags_to_movies, axis=1)

#for index, row in tags_df.iterrows():
#    if index % 10000 == 0:
#        print(index/10000)
#    tag = row["tag"]
#    if tag not in relevant_tags:
#        continue
#    movieId = row["movieId"]
#    movies_df.loc[movieId, tag] += 1

0          None
1          None
2          None
3          None
4          None
           ... 
1093355    None
1093356    None
1093357    None
1093358    None
1093359    None
Length: 1093360, dtype: object

In [136]:
movies_df2

,sci-fi,atmospheric,action,comedy,surreal,funny,based on a book,twist ending,visually appealing,romance,...,walked out/didn't finish,warlock,watch-again,well-written,william friedkin,william holden,women's lib,worst movie ever,writers at work,yahoo top pick
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,1.0,20.0,0.0,30.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
209159,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
209163,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [137]:
movies_df2["funny"]

movieId
1         30.0
2          0.0
3          1.0
4          0.0
5          0.0
          ... 
209157     0.0
209159     0.0
209163     0.0
209169     0.0
209171     0.0
Name: funny, Length: 62423, dtype: float64

In [138]:
movies_df2["max"] = movies_df2.max(axis=1)
novi = movies_df2[movies_df2["max"] > 0]
novi = novi.div(novi["max"], axis=0)
novi = novi.drop("max", axis=1)
novi

,sci-fi,atmospheric,action,comedy,surreal,funny,based on a book,twist ending,visually appealing,romance,...,walked out/didn't finish,warlock,watch-again,well-written,william friedkin,william holden,women's lib,worst movie ever,writers at work,yahoo top pick
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.008772,0.0,0.008772,0.175439,0.0,0.263158,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.058824,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.000000,0.000000,0.0,0.500000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.0,0.000000,0.333333,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208813,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
208933,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
209035,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [139]:
movies_df2

,sci-fi,atmospheric,action,comedy,surreal,funny,based on a book,twist ending,visually appealing,romance,...,warlock,watch-again,well-written,william friedkin,william holden,women's lib,worst movie ever,writers at work,yahoo top pick,max
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,1.0,20.0,0.0,30.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,114.0
2,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
209159,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
209163,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [140]:
movies_df2 = novi
movies_df1 = movies_df1.drop(["genres", "title"], axis=1)
movies_df1

,Musical,Comedy,War,Mystery,Romance,Children,Sci-Fi,Adventure,Crime,Western,Animation,Horror,Fantasy,Action,Documentary,Thriller,Film-Noir,Drama
movieId,,,,,,,,,,,,,,,,,,
1,0,1,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0
2,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0
3,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209157,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
209159,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
209163,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [143]:
movies_df = pd.concat([movies_df1, movies_df2], axis=1)
movies_df = movies_df.dropna()
movies_df

,Musical,Comedy,War,Mystery,Romance,Children,Sci-Fi,Adventure,Crime,Western,...,walked out/didn't finish,warlock,watch-again,well-written,william friedkin,william holden,women's lib,worst movie ever,writers at work,yahoo top pick
movieId,,,,,,,,,,,,,,,,,,,,,
1,0,1,0,0,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,1,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208813,0,0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
208933,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
209035,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [144]:
#few minutes
movies_df.to_csv("movie_features.csv")